## Pipeline Data Transform Tutorial 

### install

`Pipeline` is distributed along with [fate_client](https://pypi.org/project/fate-client/).

```bash
pip install fate_client
```

To use Pipeline, we need to first specify which `FATE Flow Service` to connect to. Once `fate_client` installed, one can find an cmd enterpoint name `pipeline`:

In [ ]:
!pipeline --help

Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [2]:
!pipeline init --ip 127.0.0.1 --port 9380

Pipeline configuration succeeded.


### transform data

 Before start a modeling task, local data to be used should be transformed into dataframe. 
 Typically, a party is usually a cluster which include multiple nodes. Thus, when we upload these data, the data will be allocated to those nodes.

In [1]:
from fate_client.pipeline import FateFlowPipeline

Make a `pipeline` instance:

    - initiator: 
        * role: local
        * party: 0
    - roles:
        * local: 0

note that for
    

In [3]:
pipeline = FateFlowPipeline().set_roles(local="0")
pipeline.set_site_role("local")
pipeline.set_site_party_id("0");

Define partitions for data storage

In [6]:
meta = {'delimiter': ',',
        'dtype': 'float32',
        'input_format': 'dense',
        'label_type': 'int32',
        'label_name': 'y',
        'match_id_name': 'id',
        'match_id_range': 0,
        'tag_value_delimiter': ':',
        'tag_with_value': False,
        'weight_type': 'float32'}

Define data meta, note that for local file comes with sample id, `sample_id_name` should also be specified.

In [7]:
partitions = 4

Define table name and namespace, which will be used in FATE job configuration

In [8]:
data_guest = {"name": "breast_hetero_guest", "namespace": f"experiment"}

Now, we add data to be uploaded

We can then start transforming data. Function `transform_local_file_to_dataframe` will initialize two tasks: the first task uploads local data, while the second transforms uploaded data into dataframe usable by FATE tasks.

In [ ]:
import os
data_base = "work_space/FATE/"
pipeline.transform_local_file_to_dataframe(file=os.path.join(data_base, "examples/data/breast_hetero_guest.csv"),
                                           meta=meta, head=True, extend_sid=True,
                                           name=data_guest["name"],             # name
                                           namespace=data_guest["namespace"],   # namespace
                                           partitions=partitions)